# Selección del set de datos y la elección de las columnas a emplear

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import xgboost as xgb
from sklearn.metrics import mean_squared_error
paleta_colores = sns.color_palette()
plt.style.use("fivethirtyeight")
import os
os.environ["PATH"] += os.pathsep + 'C:/Program Files/Graphviz/bin'
from openpyxl import load_workbook
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from scipy.stats import pearsonr
control = 10
workbook = load_workbook(filename=r'C:\Users\harol\OneDrive\Documents\recoleccion_data.xlsx') 
sheet = workbook.active
def crear_caracteristicas(dataframe):
        dataframe["week"] = dataframe.index.isocalendar().week
        dataframe["week"] = dataframe["week"].astype(np.int64)
        return dataframe
ciudad = ['Ji Parana', 'Manaus', 'Sao Luis', 'Parnaiba', 'Juazeiro do Norte', 'Maranguape', 'Sao Vicente', 'Sertaozinho', 'Santa Cruz do Capibaribe', 
          'Aracaju', 'Eunapolis', 'Belo Horizonte', 'Barra Mansa', 'Rio de Janeiro', 'Sao Goncalo', 'Barretos', 'Barueri', 'Guaruja',
         'Tres Lagoas', 'Rondonopolis']
for city in ciudad:
    for a in range(4):
        sheet[f'A{control + a}'] = city
        sheet[f'B{control + a}'] = "XGBoost"
        sheet[f'C{control + a}'] = "Modelo 2 - GT"
        sheet[f'D{control + a}'] = "Entrenamiento"
        sheet[f'A{control + a + 4}'] = city
        sheet[f'B{control + a + 4}'] = "XGBoost"
        sheet[f'C{control + a + 4}'] = "Modelo 2 - GT"
        sheet[f'D{control + a + 4}'] = "Test"
    if city == 'Ji Parana':
        dataframe = pd.read_excel(r'C:\Users\harol\Downloads\dataverse_files\Dengue_selected_cities.xlsx', sheet_name = city, usecols = ["date", "N_cases", "a dengue", "sintomas da dengue", "sintomas dengue", "sobre a dengue", "sintomas de dengue", "Mosquito da dengue", "Mosquito Dengue", "dengue hemorragica", "mosquito", "mosquitos" ])
        dataframe.columns = [ "week", "N_casos", "a dengue", "sintomas da dengue", "sintomas dengue", "sobre a dengue", "sintomas de dengue", "Mosquito da dengue", "Mosquito Dengue", "dengue hemorragica", "mosquito", "mosquitos" ]
    else:
        dataframe = pd.read_excel(r'C:\Users\harol\Downloads\dataverse_files\Dengue_selected_cities.xlsx', sheet_name = city, usecols = ["date", "N_cases","dengue", "a dengue", "sintomas da dengue", "sintomas dengue", "sobre a dengue", "sintomas de dengue", "Mosquito da dengue", "Mosquito Dengue", "dengue hemorragica", "mosquito", "mosquitos" ])
        dataframe.columns = [ "week", "N_casos", "dengue", "a dengue", "sintomas da dengue", "sintomas dengue", "sobre a dengue", "sintomas de dengue", "Mosquito da dengue", "Mosquito Dengue", "dengue hemorragica", "mosquito", "mosquitos" ]
    dataframe = dataframe.set_index("week")
    dataframe = dataframe.loc[ dataframe.index > "2011-01-01"]
    
    dataframe = crear_caracteristicas(dataframe)
    df_train, df_test = train_test_split(dataframe, test_size=14, shuffle=False)
    if city == 'Ji Parana':
         X_train, y_train = df_train[["a dengue", "sintomas da dengue", "sintomas dengue", "sobre a dengue",
                      "sintomas de dengue", "Mosquito da dengue", "Mosquito Dengue", "dengue hemorragica", "mosquito", 
                      "mosquitos"]].values, df_train['N_casos'].values.T
        
         X_test, y_test = df_test[["a dengue", "sintomas da dengue", "sintomas dengue", "sobre a dengue",
                      "sintomas de dengue", "Mosquito da dengue", "Mosquito Dengue", "dengue hemorragica", "mosquito", 
                      "mosquitos"]].values, df_test['N_casos'].values.T
    else:
         X_train, y_train = df_train[["dengue", "a dengue", "sintomas da dengue", "sintomas dengue", "sobre a dengue",
                      "sintomas de dengue", "Mosquito da dengue", "Mosquito Dengue", "dengue hemorragica", "mosquito", 
                      "mosquitos"]].values, df_train['N_casos'].values.T
    
         X_test, y_test = df_test[["dengue", "a dengue", "sintomas da dengue", "sintomas dengue", "sobre a dengue",
                      "sintomas de dengue", "Mosquito da dengue", "Mosquito Dengue", "dengue hemorragica", "mosquito", 
                      "mosquitos"]].values, df_test['N_casos'].values.T
        
    model = xgb.XGBRegressor( n_estimators = 10000, 
                                     max_depth = 7, 
                                     learning_rate = 0.005,
                                    early_stopping_rounds = 10,
                                    num_parallel_tree = 4
                                   )
    model.fit(X_train, y_train,
                      eval_set = [ (X_train, y_train),(X_test, y_test) ], 
                      verbose = 1000)
    predictions = model.predict(X_test)
    mse = mean_squared_error(y_test, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test, predictions) if r2_score(y_test, predictions) >= 0 else 0.0
    sheet[f'E{control + 4}'] = "RMSE"
    sheet[f'F{control + 4}'] = rmse
    sheet[f'E{control + 5}'] = "R-RMSE"
    sheet[f'F{control + 5}'] = rmse/(np.max(y_test) - np.min(y_test))
    sheet[f'E{control + 6}'] = "R^2"
    sheet[f'F{control + 6}'] = r2
    sheet[f'E{control + 7}'] = "C. Pearson"
    sheet[f'F{control + 7}'] = pearsonr(y_test, predictions)[0]
    predictions = model.predict(X_train)
    mse = mean_squared_error(y_train, predictions)
    rmse = np.sqrt(mse)
    r2 = r2_score(y_train, predictions) if r2_score(y_train, predictions) >= 0 else 0.0
    sheet[f'E{control}'] = "RMSE"
    sheet[f'F{control}'] = rmse
    sheet[f'E{control + 1}'] = "R-RMSE"
    sheet[f'F{control + 1}'] = rmse/(np.max(y_train) - np.min(y_train))
    sheet[f'E{control + 2}'] = "R^2"
    sheet[f'F{control + 2}'] = r2
    sheet[f'E{control + 3}'] = "C. Pearson"
    sheet[f'F{control + 3}'] = pearsonr(y_train, predictions)[0]
    workbook.save(filename=r'C:\Users\harol\OneDrive\Documents\recoleccion_data.xlsx')
    control += 24
    print(city)


[0]	validation_0-rmse:17.48076	validation_1-rmse:0.50382
[10]	validation_0-rmse:16.84164	validation_1-rmse:0.54131
Ji Parana
[0]	validation_0-rmse:865.24167	validation_1-rmse:137.72305
[563]	validation_0-rmse:119.48662	validation_1-rmse:47.08348
Manaus
[0]	validation_0-rmse:80.52300	validation_1-rmse:27.99227
[68]	validation_0-rmse:61.50269	validation_1-rmse:21.22631
Sao Luis
[0]	validation_0-rmse:13.20040	validation_1-rmse:4.11692
[94]	validation_0-rmse:9.90894	validation_1-rmse:3.28533
Parnaiba
[0]	validation_0-rmse:29.16015	validation_1-rmse:9.72952
[163]	validation_0-rmse:17.80325	validation_1-rmse:6.91337
Juazeiro do Norte
[0]	validation_0-rmse:7.88252	validation_1-rmse:11.32705
[283]	validation_0-rmse:2.76453	validation_1-rmse:9.35164
Maranguape
[0]	validation_0-rmse:79.70149	validation_1-rmse:5.15655
[388]	validation_0-rmse:19.24739	validation_1-rmse:3.00932
Sao Vicente
[0]	validation_0-rmse:57.42041	validation_1-rmse:0.84618
[12]	validation_0-rmse:54.72291	validation_1-rmse:1.7